## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = "sphinx_gallery"
import plotly.express as px
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import statsmodels.formula.api as smf
from leap.data_generation.utils import heaviside
from leap.data_generation.antibiotic_data import load_birth_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# Plot Settings
pio.templates.default = "plotly_white"
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'scale': 2
    }
}

## Data

Since the antibiotic dosing data is private, we will create here a mock dataset. In this dataset,
we will have the *total* number of courses of antibiotics prescribed to all infants in BC,
stratified by sex and year.

In [ ]:
# Data was collected for the years 2000-2018
years = np.arange(2000, 2018)

# Mock data for antibiotic courses prescribed to infants using an approximate linear trend
n_abx_female = [-640 * i + 16000 for i, _ in enumerate(years)]
n_abx_male = [-880 * i + 22000 for i, _ in enumerate(years)]
n_abx = n_abx_female + n_abx_male

# Create a DataFrame for antibiotic courses prescribed to infants
df_abx = pd.DataFrame(
    data=list(itertools.product(["F", "M"], years)),
    columns=["sex", "year"]
)
df_abx["n_abx"] = n_abx
df_abx.head()

,sex,year,n_abx
0,F,2000,16000
1,F,2001,15360
2,F,2002,14720
3,F,2003,14080
4,F,2004,13440


Next, since the antibiotic data measures the number of antibiotics prescribed at a population level,
we need to use population data from `Statistics Canada` to determine the number of antibiotics
prescribed per capita:

In [55]:
df_birth = load_birth_data()
df_birth.head()

/Users/work/Coding/RESP/leap/leap/data_generation/antibiotic_data.py:78: DtypeWarning:

Columns (12,14) have mixed types. Specify dtype option on import or set low_memory=False.



,year,province,sex,n_birth
0,2000,BC,M,21314
1,2000,BC,F,20119
2,2001,BC,M,20658
3,2001,BC,F,19578
4,2002,BC,M,20496


We can now merge the `df_abx` and `df_birth` dataframes:

In [ ]:
df = pd.merge(df_abx, df_birth, on=["year", "sex"], how="left")
df.head()

In [98]:
fig = px.line(
    df,
    x="year",
    y="n_abx",
    facet_col="sex",
    color="sex",
    labels={"n_abx": "Total Number of Antibiotic Courses Prescribed to Infant Population"},
)
fig.update_xaxes(title_text="Birth Year", nticks=10)
fig.update_yaxes(title_font=dict(size=12))

fig.show()

## Model

In [ ]:
df["sex"] = df.apply(lambda x: 1 if x["sex"] == "F" else 2, axis=1)

In [57]:
formula = "n_abx ~ year + sex + heaviside(year, 2005) * year"

In [58]:
θ = 800
alpha = 1 / θ

# Fit the GLM model
model = smf.glm(
    formula=formula,
    data=df,
    family=sm.families.NegativeBinomial(alpha=alpha),
    offset=np.log(df["n_birth"])
)
results = model.fit(maxiter=1000)

print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  n_abx   No. Observations:                   36
Model:                            GLM   Df Residuals:                       31
Model Family:        NegativeBinomial   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -260.35
Date:                Tue, 12 Aug 2025   Deviance:                       18.039
Time:                        13:53:48   Pearson chi2:                     18.0
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

Now that the model has been trained, we can obtain the $\beta$ parameters:

In [59]:
β_0 = results.params["Intercept"]
β_t = results.params["year"]
β_s = results.params["sex"]
β_h = results.params["heaviside(year, 2005)"]
β_th = results.params["heaviside(year, 2005):year"]

## Example: Sex = Male, Year = 2008

In [60]:
sex = 2
year = 2008
η = β_0 + β_s * sex + β_t * year + β_h * heaviside(year, 2005) + β_th * heaviside(year, 2005) * year

Now, $\eta$ is the number of antibiotics prescribed per number of infants; what we want to know, is
given an age, birth year, and sex, how many antibiotics were prescribed to that person during
infancy? To do so, we need to use the `Negative Binomial` distribution.
First, we compute $\mu$, using the `link function`.

In [61]:
µ = np.exp(η)

Next, we make sure that $\mu > 0.05$:

In [101]:
µ = max(µ, 0.05)
print(f"µ = {µ:.6f}")

µ = 0.674037


The standard parametrization of the Negative Binomial function uses $p$ and $r$, not $\mu$.
So, we need to convert $\mu$ to $p$:

In [103]:
p = θ / (θ + µ)
r = θ
print(f"p = {p:.6f}, r = {r:.4f}")

p = 0.999158, r = 800.0000


The `Negative Binomial` distribution gives us the probability of an infant being prescribed $k$
courses of antibiotics during infancy:

$$
\begin{aligned}
P(Y = k; r, p) := \binom{k+r-1}{k}(1-p)^k p^r
\end{aligned}
$$

In [70]:
n_abx = np.random.negative_binomial(r, p)
sex = "female" if sex == 1 else "male"
print(f"Number of courses of antibiotics prescribed during infancy to a {sex} person born in {year}: {n_abx}")

Number of courses of antibiotics prescribed during infancy to a male person born in 2008: 1


The number of antibiotics obtained is just a sampling from the distribution; if we repeated the
selection multiple times, we could get different values. To visualize this, we can look at the
distribution:

In [ ]:
n_abx = np.random.negative_binomial(r, p, size=100000)
fig = px.histogram(
    n_abx,
    nbins=10,
    title=f"Distribution of Antibiotic Courses Prescribed in Infancy<br>Sex = {sex}, Birth Year = {year}"
)
fig.update_xaxes(title_text="Number of Courses of Antibiotics")
fig.update_yaxes(title_text="Count")
fig.update_layout(
    legend_title_text="",
    title_x=0.5,
    title_y=0.95,
    margin=dict(t=50), # Adjust top margin for title
    showlegend=False
)
fig.show(config=config)

## Example: Visualize Multiple Years / Sexes

The example above was just the distribution for a single year and sex; we can visualize
multiple years / sexes together:

In [ ]:
def compute_abx_distribution_parameters(sex, year, β_0, β_s, β_t, β_h, β_th, θ):
    η = (
        β_0 + β_s * sex + β_t * year +
        β_h * heaviside(year, 2005) +
        β_th * heaviside(year, 2005) * year
    )
    µ = np.exp(η)
    µ = max(µ, 0.05)  # Ensure µ is greater than 0.05
    p = θ / (θ + µ)
    return p
    

Let's create a dataframe with even years from `2000` to `2018`:

In [183]:
sexes = [1, 2]
years = np.arange(2000, 2018, step=2)
df_pred = pd.DataFrame(
    data=list(itertools.product(sexes, years)),
    columns=["sex", "year"],
    dtype='Int64'
)
df_pred.head()

,sex,year
0,1,2000
1,1,2002
2,1,2004
3,1,2006
4,1,2008


Next, let's find $p$ and $r$ for each year / sex combination:

In [185]:
df_pred["p"] = df_pred.apply(
    lambda x: compute_abx_distribution_parameters(x["sex"], x["year"], β_0, β_s, β_t, β_h, β_th, θ),
    axis=1
)
df_pred["r"] = df_pred.apply(
    lambda x: r,
    axis=1
)
df_pred.head()

,sex,year,p,r
0,1,2000,0.998991,800
1,1,2002,0.999064,800
2,1,2004,0.999132,800
3,1,2006,0.999236,800
4,1,2008,0.999350,800


We now create a second dataframe, which we will use to sample the `Negative Binomial`
distribution.

In [186]:
df_dist = pd.DataFrame(
    data=list(itertools.product(sexes, years, np.zeros(1000))),
    columns=["sex", "year", "nabx"],
    dtype=int
)
df_dist = pd.merge(df_dist, df_pred, on=["year", "sex"])
df_dist.head()

,sex,year,nabx,p,r
0,1,2000,0,0.998991,800
1,1,2000,0,0.998991,800
2,1,2000,0,0.998991,800
3,1,2000,0,0.998991,800
4,1,2000,0,0.998991,800


In [189]:
grouped_df = df_dist.groupby(["sex", "year"])
df_dist["nabx"] = grouped_df.apply(
    lambda x: np.random.negative_binomial(x["r"], x["p"], size=len(x)), include_groups=False
).explode().astype(int).values
df_dist.head()

,sex,year,nabx,p,r
0,1,2000,1,0.998991,800
1,1,2000,2,0.998991,800
2,1,2000,1,0.998991,800
3,1,2000,3,0.998991,800
4,1,2000,1,0.998991,800


Now we can visualize our results:

In [ ]:
color_map={
    2: "#09bfc4",
    1: "#f39c12",
}

fig = px.histogram(
    df_dist,
    x="nabx",
    facet_col="year",
    facet_col_wrap=3,
    color="sex",
    nbins=10,
    title=f"Distribution of Antibiotic Courses Prescribed in Infancy",
    height=1000,
    width=850,
    facet_row_spacing=0.1,
    color_discrete_map=color_map,
    barmode="group"
)
fig.for_each_trace(lambda t: t.update(name="female" if t.name == "1" else "male"))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("year=", "Year: ")))
fig.update_xaxes(
    title_text="Number of Courses of Antibiotics",
    nticks=10,
    row=1,
    title=dict(font=dict(size=12))
)
fig.update_xaxes(showticklabels=True, nticks=10)
fig.update_yaxes(title_text="Count", col=1)
fig.update_layout(
    legend_title_text="",
    title_x=0.5,
    title_y=0.95,
    margin=dict(t=150), # Adjust top margin for title
    showlegend=True
)
fig.show(config=config)